# Forecasting

This page is largely used as a sandbox for creating functions and tools to achieve a streamlined method of pulling data from Fred API, performing feature engineering, and ultimately building out predictions to be able to forecast and prove effective forecasts accuracy.

Below I import some packages that I will be using to build these functions out.

In [1]:
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
import requests
import pandas as pd
import numpy as np
import plotly.express as px
from sklearn.preprocessing import StandardScaler
import polars as pl

Now I will import the functions that I have already made (and hopefully already effectively documented). Currently I have the following:
- fred_req(api_key, series_id): gathers data on a given series in it's raw format.
- fred_series_details(api_key, series_id): gathers metadata on a series.
- full_fred_dataframe(api_key, series_id_list): provides an outer-joined dataframe of all desired variables.

In [1]:
from main import fred_req, fred_series_details, full_fred_dataframe, get_prices_daily, daily_stock_df, day_to_month_col, monthly_stock_df

In [4]:
ticker = "XLK"




In [2]:
monthly_stock_df("XLK")

date,first_volume,last_volume,min_volume,max_volume,mean_volume,median_volume,sum_volume,quantile_25_volume,quantile_75_volume,first_open,last_open,min_open,max_open,mean_open,median_open,sum_open,quantile_25_open,quantile_75_open,first_close,last_close,min_close,max_close,mean_close,median_close,sum_close,quantile_25_close,quantile_75_close,first_high,last_high,min_high,max_high,mean_high,median_high,sum_high,quantile_25_high,quantile_75_high,first_low,last_low,min_low,max_low,mean_low,median_low,sum_low,quantile_25_low,quantile_75_low,first_H-L,last_H-L,min_H-L,max_H-L,mean_H-L,median_H-L,sum_H-L,quantile_25_H-L,quantile_75_H-L,first_C-O,last_C-O,min_C-O,max_C-O,mean_C-O,median_C-O,sum_C-O,quantile_25_C-O,quantile_75_C-O
date,i64,i64,i64,i64,f64,f64,i64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
1998-12-01,300500,77200,77200,338300,212628.571429,243700.0,1488400,123800.0,309300.0,32.40625,32.75,32.40625,33.1875,32.790179,32.75,229.53125,32.75,32.9375,32.046875,32.625,32.046875,32.875,32.654018,32.75,228.578125,32.6875,32.8125,32.5,32.90625,32.5,33.1875,32.910714,32.90625,230.375,32.8125,33.140625,31.78125,32.34375,31.78125,32.59375,32.359375,32.5,226.515625,32.34375,32.578125,0.71875,0.5625,0.25,0.71875,0.551339,0.5625,3.859375,0.5625,0.65625,-0.359375,-0.125,-0.40625,0.0625,-0.136161,-0.125,-0.953125,-0.125,0.0625
1999-01-01,650600,933600,295200,1492900,758331.578947,650600.0,14408300,532800.0,946600.0,32.65625,37.375,32.0,37.375,35.154605,35.5,667.9375,34.5,36.25,33.0,37.8125,33.0,37.8125,35.222862,34.875,669.234375,34.734375,36.0,33.5625,37.84375,33.5625,37.84375,35.744243,35.5625,679.140625,35.03125,36.8125,32.59375,36.9375,32.0,36.9375,34.632401,34.484375,658.015625,34.265625,35.75,0.96875,0.90625,0.53125,3.15625,1.111842,1.015625,21.125,0.953125,1.1875,0.34375,0.4375,-1.25,2.25,0.068257,0.171875,1.296875,-0.5625,0.515625
1999-02-01,913000,2342600,913000,2917400,1.9880e6,2.0655e6,37772500,1.6736e6,2.5005e6,38.15625,35.0625,33.75,38.15625,35.735197,35.75,678.96875,34.65625,36.375,37.84375,34.0625,33.84375,37.84375,35.412007,35.125,672.828125,34.625,36.1875,38.375,35.125,34.40625,38.375,36.172697,36.03125,687.28125,35.40625,36.5,37.5625,33.75,33.34375,37.5625,34.911184,34.59375,663.3125,34.25,35.5625,0.8125,1.375,0.71875,1.875,1.261513,1.3125,23.96875,1.15625,1.40625,-0.3125,-1.0,-1.5625,0.96875,-0.323191,-0.28125,-6.140625,-0.796875,0.15625
1999-03-01,1092100,774700,321400,3199100,1.0632e6,836000.0,24454200,616700.0,1.4986e6,33.625,37.5,33.5,37.5,35.433424,35.5,814.96875,35.1875,35.84375,33.90625,36.59375,33.0,37.125,35.328804,35.5,812.5625,34.9375,35.96875,34.1875,37.8125,33.71875,37.8125,35.799592,35.84375,823.390625,35.296875,36.375,33.4375,36.53125,32.875,36.75,34.874321,35.03125,802.109375,34.265625,35.5625,0.75,1.28125,0.53125,1.5,0.925272,0.84375,21.28125,0.75,1.0625,0.28125,-0.90625,-1.1875,0.875,-0.10462,0.0,-2.40625,-0.25,0.28125
1999-04-01,318500,760000,244500,1699200,704919.047619,606900.0,14803300,413000.0,1.0035e6,37.5,37.75,34.8125,39.5,37.780506,37.75,793.390625,37.5,38.5625,37.171875,36.8125,34.4375,39.0625,37.520833,37.8125,787.9375,36.875,38.453125,37.53125,37.75,35.375,39.5625,38.123512,38.40625,800.59375,37.53125,39.0,36.5,35.9375,34.125,38.6875,36.879464,37.25,774.46875,36.09375,37.875,1.03125,1.8125,0.671875,2.46875,1.244048,1.25,26.125,0.96875,1.3125,-0.328125,-0.9375,-2.125,0.9375,-0.259673,-0.125,-5.453125,-0.8125,0.375
1999-05-01,201600,132900,132900,1434900,428825.0,363000.0,8576500,276000.0,443100.0,37.0,36.46875,36.4375,38.875,37.486719,37.328125,749.734375,36.96875,37.875,37.125,36.9375,35.984375,38.625,37.377344,37.3125,747.546875,36.75,38.0,37.25,36.9375,36.75,39.28125,37.838281,37.75,756.765625,37.25,38.25,36.59375,36.15625,35.375,38.46875,36.890625,36.851562,737.8125,36.25,37.5,0.65625,0.78125,0.578125,1.5625

Below is the API Key I will be using. 

*Note: I should find a way to "hide" this so I can still post this all on Github*.

In [ ]:
api_key = '4447961bacaaa3cd54858e4994a7006e'

Here are the variables I am starting off with, but I can easily change this to get different outputs. One of the first things I want to mess with is how can I capitalize on data that is represented at different frequencies? For instance, if I want to construct a forecast that looks at the next 6 months, (i.e., looking at month intervals), how can I utilize the data shown in daily, quarterly, and yearly frequencies?

In [ ]:
list_variables = [
'JTSJOL',
'UNRATE',
'T10YIE',
'GNPCA',
]

In [ ]:
df = pl.DataFrame(fred_req(api_key, list_variables[0]), schema=[("date", str), ("value", str)])\
                .with_columns((pl.col("value").cast(pl.Float32)).alias("value"))\
                .with_columns((pl.col("date").str.to_date("%Y-%m-%d")))
df.head()

date,value
date,f32
2000-12-01,5088.0
2001-01-01,5234.0
2001-02-01,5097.0
2001-03-01,4762.0
2001-04-01,4615.0


In [ ]:
df2 = pl.DataFrame(fred_req(api_key, list_variables[1]), schema=[("date", str), ("value", str)])\
                .with_columns((pl.col("value").cast(pl.Float32)).alias("value"),\
                                (pl.col("date").str.to_date("%Y-%m-%d")))

df2.head()

date,value
date,f32
1948-01-01,3.4
1948-02-01,3.8
1948-03-01,4.0
1948-04-01,3.9
1948-05-01,3.5


In [ ]:
df = full_fred_dataframe(api_key, list_variables)

In [ ]:
def day_to_month(df:pl.DataFrame, col_name:str)->pl.DataFrame:
    """Takes in a polars dataframe and designated column name
    to perform the necessary row operations to summarize daily
    values into monthly representations.

    Args:
        df (pl.DataFrame): full polars dataframe
        col_name (str): daily column for transformation

    Returns:
        pl.DataFrame: grouped object with summary stats
    """
    # making sure data is sorted for new values
    day_col = df[["date", col_name]].sort(by="date")
    # changing date to monthly representation
    day_col = day_col.with_columns(pl.col("date").dt.strftime("%Y-%m").str.to_date("%Y-%m")).drop_nulls()
    # Perform the aggregation
    day_col = day_col.group_by("date").agg(pl.col(col_name).first().name.prefix("first_"),
                                            pl.col(col_name).last().name.prefix("last_"),
                                            pl.col(col_name).min().name.prefix("min_"),
                                            pl.col(col_name).max().name.prefix("max_"),
                                            pl.col(col_name).mean().name.prefix("mean_"),
                                            pl.col(col_name).median().name.prefix("median_"),
                                            pl.col(col_name).sum().name.prefix("sum_"),
                                            pl.col(col_name).quantile(.25).name.prefix("quantile_25_"),
                                            pl.col(col_name).quantile(.75).name.prefix("quantile_75_"),
                                            ).sort("date")
    
    return day_col

In [ ]:
import functools as ft

def monthly_df_transformations(df):

    new_dfs = []

    for col_name in list(df.columns):
        if "Daily" in col_name:
            new_dfs.append(day_to_month(df, col_name))
        elif col_name=="date":
            pass
        else:
            new_dfs.append(df[["date", col_name]].drop_nulls())

    df_joined = ft.reduce(lambda left, right: left.join(right, on='date', how='outer_coalesce'), new_dfs)

    return df_joined

In [ ]:
monthly_df_transformations(df)

date,Job Openings: Total Nonfarm (Monthly),Unemployment Rate (Monthly),first_10-Year Breakeven Inflation Rate (Daily),last_10-Year Breakeven Inflation Rate (Daily),min_10-Year Breakeven Inflation Rate (Daily),max_10-Year Breakeven Inflation Rate (Daily),mean_10-Year Breakeven Inflation Rate (Daily),median_10-Year Breakeven Inflation Rate (Daily),sum_10-Year Breakeven Inflation Rate (Daily),quantile_25_10-Year Breakeven Inflation Rate (Daily),quantile_75_10-Year Breakeven Inflation Rate (Daily),Real Gross National Product (Annual)
date,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32
2003-04-01,108.0,0.0,0.82,0.73,0.0,0.83,0.739545,0.765,16.27,0.74,0.8,null
2003-05-01,289.0,0.1,0.75,0.6,0.0,0.75,0.630909,0.66,13.879999,0.61,0.69,null
2003-07-01,981.0,0.2,0.64,0.08,0.0,0.98,0.523913,0.63,12.050001,0.08,0.88,null
2003-08-01,190.0,0.1,0.04,0.16,0.0,0.98,0.172381,0.16,3.62,0.07,0.19,null
2003-09-01,91.0,0.1,0.0,0.01,0.0,0.2,0.084091,0.055,1.85,0.03,0.16,null
2003-10-01,305.0,0.0,0.0,0.4,0.0,0.4,0.204783,0.23,4.71,0.1,0.32,null
2003-12-01,414.0,0.7,0.32,0.27,0.0,0.37,0.271739,0.28,6.249999,0.24,0.33,null
2004-01-01,424.0,0.7,0.0,0.31,0.0,0.35,0.239091,0.265,5.26,0.22,0.31,5542.707031
2004-03-01,525.0,0.8,0.45,0.38,0.24,0.48,0.361304,0.35,8.309999,0.32,0.39,null


In [ ]:
col_name = "10-Year Breakeven Inflation Rate (Daily)"
day_col = day_to_month(df, col_name)